In [1]:
import tensorflow as tf
# inceptionv3 imports
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

#callback imports
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Create directories
from pathlib import Path

original_dataset_path = Path('new_dataset/')

base_path = Path('new_dataset/')
base_path.mkdir(parents=True, exist_ok=True)

train_path = base_path / 'train'
train_path.mkdir(parents=True, exist_ok=True)

validation_path = base_path / 'val'
validation_path.mkdir(parents=True, exist_ok=True)

test_path = base_path / 'test'
#test_path.mkdir(parents=True, exist_ok=True)

In [3]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(4, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', 
              loss='categorical_crossentropy', 
              metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.CategoricalAccuracy()])

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    str(train_path),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    str(validation_path),
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 8860 images belonging to 4 classes.
Found 20 images belonging to 4 classes.


In [5]:
#callbacks for first learning stage
tensorboard_callback = TensorBoard(log_dir="./logs/stage1")
filepath = "saved-model-{epoch:02d}-stage1.hdf5"
checkpoint_callback = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')

In [6]:
# TODO = validation_split=0.30 ?
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=validation_generator, 
    validation_steps=len(validation_generator),
    callbacks=[tensorboard_callback, checkpoint_callback]
)

Epoch 1/5
277/277 [==============================] - ETA: 0s - loss: 0.3158 - accuracy: 0.0367 - categorical_accuracy: 0.9503
Epoch 00001: saving model to saved-model-01-stage1.hdf5
277/277 [==============================] - 212s 764ms/step - loss: 0.3158 - accuracy: 0.0367 - categorical_accuracy: 0.9503 - val_loss: 0.2449 - val_accuracy: 0.0125 - val_categorical_accuracy: 0.9000
Epoch 2/5
277/277 [==============================] - ETA: 0s - loss: 0.0631 - accuracy: 0.1554 - categorical_accuracy: 0.9894
Epoch 00002: saving model to saved-model-02-stage1.hdf5
277/277 [==============================] - 216s 781ms/step - loss: 0.0631 - accuracy: 0.1554 - categorical_accuracy: 0.9894 - val_loss: 0.5616 - val_accuracy: 0.0875 - val_categorical_accuracy: 0.8500
Epoch 3/5
277/277 [==============================] - ETA: 0s - loss: 0.0375 - accuracy: 0.2028 - categorical_accuracy: 0.9957
Epoch 00003: saving model to saved-model-03-stage1.hdf5
277/277 [==============================] - 209s 755m

KeyboardInterrupt: 

In [7]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy', 'categorical_accuracy'])

0 input_1
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_14
5

In [13]:
# TODO = validation_split=0.30 ?
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=5,
    validation_data=validation_generator, 
    validation_steps=len(validation_generator),
    callbacks=[tensorboard_callback, checkpoint_callback]
)

Epoch 1/5
277/277 [==============================] - ETA: 0s - loss: 0.0576 - accuracy: 0.9827 - categorical_accuracy: 0.9827
Epoch 00001: saving model to saved-model-01-stage1.hdf5
277/277 [==============================] - 284s 1s/step - loss: 0.0576 - accuracy: 0.9827 - categorical_accuracy: 0.9827 - val_loss: 0.3702 - val_accuracy: 0.9500 - val_categorical_accuracy: 0.9500
Epoch 2/5
277/277 [==============================] - ETA: 0s - loss: 0.0087 - accuracy: 0.9977 - categorical_accuracy: 0.9977
Epoch 00002: saving model to saved-model-02-stage1.hdf5
277/277 [==============================] - 247s 891ms/step - loss: 0.0087 - accuracy: 0.9977 - categorical_accuracy: 0.9977 - val_loss: 0.4533 - val_accuracy: 0.9500 - val_categorical_accuracy: 0.9500
Epoch 3/5
277/277 [==============================] - ETA: 0s - loss: 0.0073 - accuracy: 0.9973 - categorical_accuracy: 0.9973
Epoch 00003: saving model to saved-model-03-stage1.hdf5
277/277 [==============================] - 275s 992ms/s

In [8]:
#callbacks for second learning stage
tensorboard_callback2 = TensorBoard(log_dir="./logs/stage1")
filepath2 = "saved-model-{epoch:02d}-stage2.hdf5"
checkpoint_callback2 = ModelCheckpoint(filepath2, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    str(test_path),
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical')
model.evaluate(test_generator, steps=20)
#print('test acc:', test_acc)

Found 20 images belonging to 4 classes.
20/20 [==============================] - 1s 63ms/step - loss: 0.4551 - accuracy: 0.9000 - categorical_accuracy: 0.9000


[0.4550955295562744, 0.8999999761581421, 0.8999999761581421]

In [12]:
model.save('model_best_new.tf')


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model_best_new.tf/assets
